In [1]:
import numpy as np 
import pandas as pd 
import warnings
warnings.filterwarnings("ignore")

#import src
import Cleanup
import Transformation
import FileProcessing
#define the objects to be called later. 
cleanup=Cleanup.Cleanup()
transform=Transformation.Transformation()
fileProcessing=FileProcessing.FileProcessing()



In [2]:

file_name='Next_Generation_Simulation__NGSIM__Vehicle_Trajectories_and_Supporting_Data'
ngsim=fileProcessing.read_input(file_name)

original File path: c:\Users\StudentAccount\Python\Capstone\gitcodelocation\DataDrivenCarFollowing\datadrivencarfollowing-v1\scripts
Data File path: c:\Users\StudentAccount\Python\Capstone\gitcodelocation\DataDrivenCarFollowing\datadrivencarfollowing-v1\data


In [3]:
df=ngsim

In [4]:

df_101 = df[((df['Location'] == 'us-101'))]

df_i80 = df[((df['Location'] == 'i-80'))]

4784679 duplicate values have been removed
4784791 duplicate values have been removed


4547708 duplicate values have been removed
4548135 duplicate values have been removed


In [7]:
right_df = df[['Vehicle_ID', 'Global_Time',
                'v_Vel', 'v_Acc', 'lane_changes', 'Local_Y', 'v_Class']]
right_df.rename(columns={'Vehicle_ID': 'Prec_Vehicle_ID', 'v_Vel': 'preceding_Vehicle_Velocity',
                'v_Acc': 'preceding_Vehicle_Acceleration', "lane_changes": "preceding_car_lane_changes", "Local_Y": "preceding_Local_Y", "v_Class": "preceding_v_Class"}, inplace=True)
df['Prec_Vehicle_ID'] = df['Preceding']
df = df.merge(right=right_df, how='left', on=(
    'Prec_Vehicle_ID', 'Global_Time'))
df['preceding_Vehicle_Velocity'] = df['preceding_Vehicle_Velocity'].fillna(
    0)
df['preceding_Vehicle_Acceleration'] = df['preceding_Vehicle_Acceleration'].fillna(
    0)
df['preceding_Local_Y'] = df['preceding_Local_Y'].fillna(9999)

df['preceding_car_lane_changes'] = df['preceding_car_lane_changes'].fillna(
    False)
df['preceding_v_Class'] = df['preceding_v_Class'].fillna(0)

KeyError: "['lane_changes'] not in index"

In [ ]:
df['Velocity Difference_Following-Preceding'] = df['v_Vel'] - \
    df['preceding_Vehicle_Velocity']
df['Acceleration Difference_Following-Preceding'] = df['v_Acc'] - \
    df['preceding_Vehicle_Acceleration']
df = df.sort_values(by=['Global_Time'],
                    ascending=True, ignore_index=True)
df['pair_Time_Duration'] = (df.groupby(
    ['L-F_Pair'], as_index=False).cumcount()*0.1)
x = (df[['L-F_Pair', 'pair_Time_Duration']].groupby(['L-F_Pair'],
        as_index=False).max(['pair_Time_Duration']))
dict_lenght = dict(zip(x['L-F_Pair'], x['pair_Time_Duration']))
df["total_pair_duration"] = df["L-F_Pair"].map(dict_lenght)

In [ ]:
#filtered=ngsim[(ngsim['Vehicle_ID'] == 2876) & ( ngsim['Global_Time'] == 1118847795200) ]
filtered=ngsim[(ngsim['Vehicle_ID'] == 1984) | ( ngsim['Vehicle_ID'] == 1978) ]

In [ ]:
filtered

,Vehicle_ID,Frame_ID,Total_Frames,Global_Time,Local_X,Local_Y,Global_X,Global_Y,v_length,v_Width,...,D_Zone,Int_ID,Section_ID,Direction,Movement,Preceding,Following,Space_Headway,Time_Headway,Location
2315,1984,7805,885,1118848622500,7.929,1186.954,6451959.424,1872595.451,16.5,5.9,...,NaN,NaN,NaN,NaN,NaN,1979,1995,69.89,1.98,us-101
3924,1984,7627,885,1118848604700,7.410,914.112,6451754.064,1872775.133,16.5,5.9,...,NaN,NaN,NaN,NaN,NaN,1979,1995,31.00,6.24,us-101
8570,1984,5808,647,1118847559700,6.353,655.436,6451590.846,1872919.955,12.0,4.5,...,NaN,NaN,NaN,NaN,NaN,1978,1989,37.00,2.33,us-101
11775,1984,7963,885,1118848638300,8.227,1919.969,6452511.143,1872112.855,16.5,5.9,...,NaN,NaN,NaN,NaN,NaN,1979,1995,73.93,1.48,us-101
14428,1984,5832,647,1118847562100,5.990,701.929,6451625.884,1872889.678,12.0,4.5,...,NaN,NaN,NaN,NaN,NaN,1978,1989,45.00,2.25,us-101
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11847252,1978,6320,751,1113437398900,18.138,614.889,6042773.578,2133680.159,15.8,6.4,...,NaN,NaN,NaN,NaN,NaN,1968,1988,59.99,2.78,i-80
11847823,1984,6486,939,1113437415500,66.094,816.872,6042795.425,2133886.993,17.8,7.9,...,NaN,NaN,NaN,NaN,NaN,1981,2009,54.78,2.99,i-80
11849613,1984,6605,728,1113433795400,17.112,1429.730,6042652.563,2134486.528,14.8,6.9,...,NaN,NaN,NaN,NaN,NaN,1992,1998,41.54,1.86,i-80
11849740,1984,5912,939,1113437358100,66.372,89.454,6042886.122,2133164.598,17.8,7.9,...,NaN,NaN,NaN,NaN,NaN,1956,0,69.98,2.97,i-80


In [ ]:
fileProcessing.export_file(filtered,'verify_file')

True